### Loading PACE Chlorophyll Data

Went to Nasa's Earth Data website https://search.earthdata.nasa.gov/ to download data. 

- searched 'PACE'
-  put it temporal range
-  hit downlaod all... Harmony did not work for me
-  downloaded Earthdata Download in order to create a .zip file

In [1]:
import os, re
import earthaccess
import numpy as np

In [2]:
# choose times
temporal = ("2024-06-04", "2024-11-11")
# specific day to look at
time = np.datetime64("2024-10-14")
lat_west = slice(48, 29)     
lon_west = slice(-133, -117)  




# login to nasa
auth = earthaccess.login()

List of PACE products:

https://search.earthdata.nasa.gov/search?q=pace

In [3]:
results_day = earthaccess.search_data(
    short_name = "PACE_OCI_L3M_CHL",
    temporal = temporal,
    granule_name="*.DAY.*.4km.*"
)
len(results_day)

158

In [4]:
# load the data
fileset = earthaccess.open(results_day)


QUEUEING TASKS | :   0%|          | 0/158 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/158 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/158 [00:00<?, ?it/s]

In [5]:
# We need to create the dates from the fileset since the original files do not have the time
import pandas as pd
import re

# Extract dates from file paths
date_pattern = re.compile(r"PACE_OCI\.(\d{8})")

dates = []
for f in fileset:
    match = date_pattern.search(f.path)
    if not match:
        raise ValueError(f"Could not extract date from: {f.path}")
    date_str = match.group(1)
    dates.append(pd.to_datetime(date_str, format="%Y%m%d"))

# Sanity check
if len(dates) != len(fileset):
    raise ValueError(f"Mismatch: found {len(dates)} dates for {len(fileset)} files")

dates[0:3]

[Timestamp('2024-06-04 00:00:00'),
 Timestamp('2024-06-05 00:00:00'),
 Timestamp('2024-06-06 00:00:00')]

from dataset create xarray

In [6]:
import xarray as xr
CHL = xr.open_mfdataset(
    fileset,
    combine="nested",
    concat_dim="time"
)

# Overwrite with correct dates
CHL = CHL.assign_coords(time=("time", dates))
CHL_west = CHL.sel(lat=lat_west, lon=lon_west)
CHL_west = CHL_west.chlor_a.sel(time=time, method='nearest').where(lambda x: x > 0)  # drop zeros for LogNorm
CHL_west = CHL_west.clip(min=0.01)

In [7]:
CHL_west

<xarray.DataArray 'chlor_a' (lat: 456, lon: 384)> Size: 700kB
dask.array<clip, shape=(456, 384), dtype=float32, chunksize=(440, 384), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 2kB 47.98 47.94 47.9 47.85 ... 29.15 29.1 29.06 29.02
  * lon      (lon) float32 2kB -133.0 -132.9 -132.9 ... -117.1 -117.1 -117.0
    time     datetime64[ns] 8B 2024-10-14
Attributes:
    long_name:      Chlorophyll Concentration, OCI Algorithm
    units:          mg m^-3
    standard_name:  mass_concentration_of_chlorophyll_in_sea_water
    valid_min:      0.001
    valid_max:      100.0
    reference:      Hu, C., Lee Z., and Franz, B.A. (2012). Chlorophyll-a alg...
    display_scale:  log
    display_min:    0.01
    display_max:    20.0

#### Trawl Data

In [10]:
catch = pd.read_csv("/contributor_folders/isidora/catch_data_extraction.csv", index_col=0)
catch


FileNotFoundError: [Errno 2] No such file or directory: '/contributor_folders/isidora/catch_data_extraction.csv'